In [22]:
import requests
import ipywidgets as widgets
import json
import time
import os
from IPython.display import display, HTML
from traitlets import Int, Unicode, observe

In [23]:
display(HTML('<link rel="stylesheet" type="text/css" href="//igv.org/web/release/1.0.6/igv-1.0.6.css">'))

In [24]:
class TabsWidget(widgets.DOMWidget):
    
    _view_name = Unicode('TabsView').tag(sync=True)
    _view_module = Unicode('tabsDemo').tag(sync=True)
    frameHeight = Int(300).tag(sync=True)
    _chromLocString = "chr1:0-0";
    _rawMessage = "";
    _incomingMessage = {};
    msgFromKernel = Unicode("{}").tag(sync=True)
    msgFromBrowser = Unicode("{}").tag(sync=True)
    status = "initial status message\n"

       # ensure that any ensuing message is seen as novel in the browser
       # the message (a json-ified string) is a mutable traitlet.  the browser only sees 
       # the message if it changes
    def _sendResetMessageToBrowser(self):  
       self.msgFromKernel = json.dumps({"cmd": "cleanSlate", "status": "nop", "callback": "", "payload": ""});

    def setHeight(self, height):
       print("setHeight(%d) "% height)
       self.frameHeight = height
        
    def getChromLocString(self):
        return self._chromLocString
    
    def ping(self):
        self._sendResetMessageToBrowser()
        self.msgFromKernel = json.dumps({"cmd": "ping", "status": "request", "callback": "", "payload": ""});

    def deleteGraph(self):
      self._sendResetMessageToBrowser()
      self.msgFromKernel = json.dumps({"cmd": "deleteGraph", "status": "request", "callback": "", "payload": ""});

    def addGraph(self, gjson):
      self._sendResetMessageToBrowser();
      self.msgFromKernel = json.dumps({"cmd": "addGraph", "status": "request",
                                       "callback": "",    "payload": gjson});


    def loadNetworkStyleFile(self, filename):
      if(not os.path.isfile(filename)):
        print("file '%s' not found" % filename)
        return;
      self._sendResetMessageToBrowser();
      self.msgFromKernel = json.dumps({"cmd": "loadNetworkStyleFile", "status": "request", "callback": "", 
                                       "payload": filename});
        

    @observe('msgFromBrowser')
    def msg_arrived(self, change):
       #self.status += "msgFromBrowser has arrived: %f\n" % time.time()
       self._rawMessage = change['new']
       #self.status += "rawMessage: %s\n" % self._rawMessage
       self._incomingMessage = json.loads(self._rawMessage)
       cmd = self._incomingMessage["cmd"]
       #self.status += "cmd: %s\n"  % cmd
       if(cmd == "updateChromLocString"):
          self._chromLocString = self._incomingMessage["payload"]
       else:
         self.status += "unrecognized cmd: '%s'" % cmd


In [25]:
display(HTML(data="""
<style>
    div#notebook-container    { width: 97%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [42]:
%%javascript
"use strict"
require.config({
    shim: {'bootstrap': {'deps' :['jquery']},
           'igv':       {'deps' :['jquery', 'jquery-ui', 'bootstrap']}
           },

    paths: {'jquery'    :   'http://code.jquery.com/jquery-1.12.4.min',
            'jquery-ui' :   'http://code.jquery.com/ui/1.12.1/jquery-ui.min',
            'cytoscape' :   'http://cytoscape.github.io/cytoscape.js/api/cytoscape.js-latest/cytoscape',
            //'cytoscape' :   'http://localhost:8099/js/cytoscape-2.7.10',
            'bootstrap' :   'http://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min',
            'igv'       :   'http://igv.org/web/release/1.0.6/igv-1.0.6'
           }
      });

require.undef('tabsDemo')

define('tabsDemo', ["jupyter-js-widgets", "jquery", "jquery-ui", "cytoscape", "igv"], 
       function(widgets, $, ui, cytoscape, igv) {
    
    var TabsView = widgets.DOMWidgetView.extend({

        initialize: function() {
           this.options = {}
           this.chromLocString = "";
           console.log("constructing TabsView");
           this.frameHeight = "800px";
           },

        resizeHandler: function(){
           console.log("TabsView resizeHandler")            
           },
        
        createMasterTabsDiv: function(){
           var masterTabsDiv = $("<div id='masterTabsDiv' style='border:1px solid gray; height: 1200px; width: 97%'></div>");
           var list = $("<ul/>");
           list.append("<li><a href='#tab-2'>igv</a></li>");
           list.append("<li><a href='#tab-1'>cytoscape</a></li>");
           list.append("<li><a href='#tab-3'>three</a></li>");
           masterTabsDiv.append(list);

           var tab2 = $("<div id='tab-1'></div>");
           tab2.append("<div id='cyDiv' style='border:1px solid blue; height:1000px;'></div>");

           var tab1 = $("<div id='tab-2'></div>");
           tab1.append("<div id='igvDiv' style='border:1px solid green; height:1000px;'></div>");
            
            
           var tab3 = $("<div id='tab-3'>contents 3</div>");
            
           masterTabsDiv.append(tab1);
           masterTabsDiv.append(tab2);
           masterTabsDiv.append(tab3);
            
           return(masterTabsDiv);
           },
         
        getCyOptions: function(){
           var value = {container: $("#cyDiv"), 
                                      elements: {nodes: [{data: {id:'a'}}],
                                                 edges: [{data:{source:'a', target:'a'}}]},
                                      style: cytoscape.stylesheet()
                                        .selector('node').style({'background-color': '#d22',
                                                                 'label': 'data(id)',
                                                                 'text-valign': 'center',
                                                                 'text-halign': 'center',
                                                                 'border-width': 1})
                                         .selector('edge').style({'line-color': 'black',
                                                                  'target-arrow-shape': 'triangle',
                                                                  'target-arrow-color': 'black',
                                                                  'curve-style': 'bezier'})
                       };
            var emptyGraphValue = {container:$("#cyDiv")};
            //return(emptyGraphValue);
            return(value);
            }, // getCyOptions
        
        
        getIgvOptions: function(){
           var igvOptionsLocal = {locus: "5:88,621,548-88,999,827",
             reference: {id: "geneSymbols_hg38",
                fastaURL: "http://localhost/data/genomes/human_g1k_v37_decoy.fasta",
                 cytobandURL: "http://localhost/data/annotations/b37_cytoband.txt"
                 },
                tracks: [
                  {name: 'Gencode v24',
                   url: "http://localhost://data/hg38/gencode.v24.annotation.sorted.gtf.gz",
                  indexURL: "http://localhost://data/hg38/gencode.v24.annotation.sorted.gtf.gz.tbi",
                  format: 'gtf',
                  visibilityWindow: 2000000,
                  displayMode: 'EXPANDED'
                  },
                 {name: 'geneSymbols_hg38',
                  url: 'http://localhost/data/hg38/geneSymbolSearch.bed',
                  indexed: false,
                  searchable: true,
                  //visibilityWindow: 5000000,
                  displayMode: 'COLLAPSED',
                  color: "#448844"
                  },
                 {name: "igap gwas",
                  url: 'http://localhost/data/hg38/variants/igap.bed',
                  indexed: false,
                  searchable: true,
                  //visibilityWindow: 5000000,
                  displayMode: 'EXPANDED',
                  color: "#884444"
                  }
                  ]
              }; // igvOptionsLocal
             return(igvOptionsLocal);
           }, // getIgvOptions
 
        render: function() {
            var multiWidget = this;
            console.log("entering render");
            this.listenTo(this.model, 'change:msgFromKernel', this.dispatchRequest, this);
            this.masterTabsDiv = this.createMasterTabsDiv();
            this.$el.append(this.masterTabsDiv);
            this.listenTo(this.model, 'change:frameHeight', this.frameDimensionsChanged, this);
            setTimeout(function(){
               window.browser = igv.createBrowser($("#igvDiv"), multiWidget.getIgvOptions());
               //window.browser.on('locuschange', function(referenceFrame, chromLocString){multiWidget.chromLocString=chromLocString});
               window.browser.on('locuschange', function(referenceFrame, chromLocString){
                   multiWidget.updateChromLocStringToKernel(multiWidget, chromLocString)});
               setTimeout(function() {
                   console.log("about to call tabs()");
                    $("#masterTabsDiv").tabs({
                       activate: function(){
                         console.log("tabs activate")
                         cy.resize();
                       }});
                   $("[href='#tab-2']").trigger("click");
                   }, 8000);
               }, 3000);
            setTimeout(function(){
               var options = multiWidget.getCyOptions();
               console.log(JSON.stringify(options));
               window.cy = cytoscape(multiWidget.getCyOptions());
               console.log("window.cy created")
               }, 1000);
             }, // render

        dispatchRequest: function(){
           console.log("dispatchRequest");
           var msgRaw = this.model.get("msgFromKernel");
           var msg = JSON.parse(msgRaw);
           console.log(msg);
           console.log("========================");
           console.log(this);
           switch(msg.cmd) {
              case 'ping':
                console.log("ping! from the kernel");
                break;
              case 'deleteGraph':
                cy.edges().remove();
                cy.nodes().remove();
                break;
              case 'loadNetworkStyleFile':
                var styleFile = msg.payload;
                this.loadNetworkStyleFile(styleFile);
                break;
              case 'addGraph':
                console.log("--- addGraph")
                var jsonGraph = msg.payload;
                console.log(jsonGraph)
                cy.json(jsonGraph);
                //cy.layout({name: "grid"});
                cy.fit(100);
                //$("#masterTabsDiv").tabs("option", "active", 1)
                $("[href='#tab-1']").trigger("click");
                break;
            default:
               console.log("unrecognized msg.cmd: " + msg.cmd);
             } // switch
           }, 

       updateChromLocStringToKernel: function(multiWidget, newString){
           console.log("xxx " + newString);
           var jsonString = JSON.stringify({cmd: "updateChromLocString",
                                            status: "request",
                                            callback: "",
                                            payload: newString});
            multiWidget.model.set("msgFromBrowser", jsonString);
            multiWidget.touch()
            },
                                                
        loadNetworkStyleFile: function(filename){
           console.log("--- entering loadNetworkStyleFile")
           var cyObj = window.cy
           var str = window.location.href;
           var url = str.substr(0, str.lastIndexOf("/")) + "/" + filename;
           url = url.replace("/notebooks/", "/files/");
           console.log("--- loadNetworkStyleFile: " +  url);
            $.getScript(url)
              .done(function(script, textStatus) {
                 console.log(textStatus);
                 cyObj.style(vizmap);
                 })
             .fail(function( jqxhr, settings, exception ) {
                console.log("getScript error trying to read " + filename);
                console.log("exception: ");
                console.log(exception);
                });
          },


        frameDimensionsChanged: function(){
           console.log("frameDimensionsChanged");
           var oldHeight = $("#mainDiv").height()
           var oldWidth = $("#mainDiv").width()
           var newHeight = this.model.get("frameHeight");
           var msg = "<center>tabs demo, height: " + oldHeight + " -> " + newHeight + "</center>";
           $("#mainDiv").html(msg);
           $("#masterTabsDiv").height(newHeight);
           }, 
    });
    return {
        TabsView: TabsView
    };
});

<IPython.core.display.Javascript object>

In [43]:
app = TabsWidget()
display(app)

In [44]:
app.deleteGraph()

In [45]:
x2 = app.getChromLocString(); print(x2)

7:101,165,144-101,166,328


In [46]:
import zmq

In [47]:
socketContext = zmq.Context()
socket = socketContext.socket(zmq.REQ)
socket.connect("tcp://localhost:%s" % '5557')
socket

In [48]:
msg = {"cmd": "ping", "status": "request", "callback": "", "payload": ""}
msg_json = json.dumps(msg)
print(msg_json)
socket.send_string(json.dumps(msg))
print(json.loads(socket.recv_string()))

{"callback": "", "payload": "", "status": "request", "cmd": "ping"}
{'callback': [''], 'payload': ['pong'], 'status': ['result'], 'cmd': {}}


msg = json.dumps({"cmd": "getTestNetwork", "status": "request", "callback": "", "payload": "VGF"})
socket.send_string(msg)
gjson = json.loads(socket.recv_string())["payload"]
gjson

app.deleteGraph()
app.addGraph(gjson)

In [49]:
region = app.getChromLocString()
msg = json.dumps({"cmd": "createGeneModel", "status": "request", "callback": "", "payload": {"targetGene": "VGF", "footprintRegion": region}})
print(msg)
socket.send_string(msg)
gjson = json.loads(socket.recv_string())["payload"][0]  # not sure why it comes in as a 1-element array

{"callback": "", "payload": {"targetGene": "VGF", "footprintRegion": "7:101,165,144-101,166,328"}, "status": "request", "cmd": "createGeneModel"}


In [50]:
print(gjson)
print(json.loads(gjson))
graphData = json.loads(gjson)
#graphData

{"elements": [ {"data": {"id": "VGF", "type": "targetGene", "label": "VGF", "distance": 0, "gene.cor": 0, "beta": 0, "purity": 0}}, {"data": {"id": "MXI1", "type": "TF", "label": "MXI1", "distance": 0, "gene.cor": -0.785143150920807, "beta": -0.785143150920807, "purity": 166.917817584712}}, {"data": {"id": "SP1", "type": "TF", "label": "SP1", "distance": 0, "gene.cor": -0.759678085572039, "beta": -0.759678085572039, "purity": 93.1854730611782}}, {"data": {"id": "EGR4", "type": "TF", "label": "EGR4", "distance": 0, "gene.cor": 0.680763550627405, "beta": 0.680763550627405, "purity": 106.351560354284}}, {"data": {"id": "TCF12", "type": "TF", "label": "TCF12", "distance": 0, "gene.cor": -0.662403672967896, "beta": -0.662403672967896, "purity": 19.6037357608875}}, {"data": {"id": "NPAS1", "type": "TF", "label": "NPAS1", "distance": 0, "gene.cor": 0.654649607771566, "beta": 0.654649607771566, "purity": 11.2310849951552}}, {"data": {"id": "MITF", "type": "TF", "label": "MITF", "distance": 0, 

In [51]:
app.deleteGraph()
app.addGraph(graphData)

In [52]:
app.loadNetworkStyleFile("style.js")

In [20]:
import os